In [62]:
# Imports

# Data import and manipulation
import pandas as pd
# Math
import numpy as np
# Let's go ahead and seed the notebook, for reproducibility
np.random.seed(113)

# Visualizing
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as offline

from viz_utils import *

# Interactive widget
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
df = pd.read_csv("data/TX_Buyers_Shipment_Location_Data.csv",
                 usecols=["REPORTER_NAME", "REPORTER_LAT", "REPORTER_LONG",
                          "BUYER_NAME", "BUYER_LAT", "BUYER_LONG",
                          "TRANSACTION_DATE", "DOSAGE_UNIT"])
df.head()

,REPORTER_NAME,BUYER_NAME,TRANSACTION_DATE,DOSAGE_UNIT,REPORTER_LAT,REPORTER_LONG,BUYER_LAT,BUYER_LONG
0,BELLCO DRUG CORP,LONGHORN DRUG CO,2006-01-11,500.0,40.69,-73.41,32.4,-94.90
1,BELLCO DRUG CORP,MALLEY'S PHARMACY,2006-01-04,500.0,40.69,-73.41,30.1,-94.17
2,BELLCO DRUG CORP,MALLEY'S PHARMACY,2006-11-02,400.0,40.69,-73.41,30.1,-94.17
3,BELLCO DRUG CORP,MALLEY'S PHARMACY,2007-01-04,100.0,40.69,-73.41,30.1,-94.17
4,BELLCO DRUG CORP,MALLEY'S PHARMACY,2007-02-14,100.0,40.69,-73.41,30.1,-94.17


In [3]:
df["TRANSACTION_DATE"] = pd.to_datetime(df["TRANSACTION_DATE"],
                                          format='%Y-%m-%d')

In [4]:
df["TRANSACTION_YEAR"] = df["TRANSACTION_DATE"].dt.year

In [5]:
df.drop(columns="TRANSACTION_DATE", inplace=True)

In [6]:
data_agg = df.groupby(["REPORTER_NAME", "REPORTER_LAT", "REPORTER_LONG",
                       "BUYER_NAME", "BUYER_LAT", "BUYER_LONG",
                       "TRANSACTION_YEAR"]).agg({"DOSAGE_UNIT": "sum"}).reset_index()

In [7]:
data_agg.head()

,REPORTER_NAME,REPORTER_LAT,REPORTER_LONG,BUYER_NAME,BUYER_LAT,BUYER_LONG,TRANSACTION_YEAR,DOSAGE_UNIT
0,"A F HAUSER, INC",41.5,-87.0,"ADAMS, NATHALIA C (DVM)",33.23,-97.11,2006,5000.0
1,"A F HAUSER, INC",41.5,-87.0,"ALLRED, JOHN A",29.70,-98.10,2006,1900.0
2,"A F HAUSER, INC",41.5,-87.0,"BOBECHKO, WALTER P MD",32.90,-96.79,2006,1500.0
3,"A F HAUSER, INC",41.5,-87.0,"BRANTLEY, GARY C DVM",32.97,-96.74,2009,700.0
4,"A F HAUSER, INC",41.5,-87.0,"BRANTLEY, GARY C DVM",32.97,-96.74,2010,200.0


In [8]:
data_agg.shape

(85658, 8)

In [39]:
data_agg["DOSAGE_UNIT"].describe()


count    8.565800e+04
mean     6.341626e+04
std      1.285731e+05
min      1.000000e+00
25%      1.400000e+03
50%      1.046000e+04
75%      7.013000e+04
max      3.830800e+06
Name: DOSAGE_UNIT, dtype: float64

In [9]:
years = list(range(2006,2013))
years

[2006, 2007, 2008, 2009, 2010, 2011, 2012]

In [10]:
# def grab_year_lat_long_weight(df):
#     max_weight = df["DOSAGE_UNIT"].max()
    
#     year_dict = {}
#     for year in years:
#         year_df = df.loc[df["TRANSACTION_YEAR"] == year]
        
#         year_lat_list = []
#         year_lon_list = []
#         year_weight_list = []
        
#         for i in year_df.index:
#             i_lat_list = [year_df["REPORTER_LAT"][i], year_df["BUYER_LAT"][i]]
#             year_lat_list.append(i_lat_list)
            
#             i_lon_list = [year_df["REPORTER_LONG"][i], year_df["BUYER_LONG"][i]]
#             year_lon_list.append(i_lon_list)
            
#             i_weight = year_df["DOSAGE_UNIT"][i] / max_weight
#             year_weight_list.append(i_weight)
        
#         year_dict[year] = [year_lat_list, year_lon_list, year_weight_list]
            
#     return year_dict

In [11]:
# lat_long_weight_dict = grab_year_lat_long_weight(data_agg)

In [12]:
# from plotly.graph_objs import *

In [13]:
# fig_dict = {
#     "data": [],
#     "layout": {},
#     "frames": []
# }

# fig_dict["layout"]["sliders"] = {
#     "args": [
#         "transition", {
#             "duration": 400,
#             "easing": "cubic-in-out"
#         }
#     ],
#     "initialValue": "2006",
#     "plotlycommand": "animate",
#     "values": years,
#     "visible": True
# }

# sliders_dict = {
#     "active": 0,
#     "yanchor": "top",
#     "xanchor": "left",
#     "currentvalue": {
#         "font": {"size": 20},
#         "prefix": "Year:",
#         "visible": True,
#         "xanchor": "right"
#     },
#     "transition": {"duration": 300, "easing": "cubic-in-out"},
#     "pad": {"b": 10, "t": 50},
#     "len": 0.9,
#     "x": 0.1,
#     "y": 0,
#     "steps": []
# }

# first_year = 2006
# first_data_dict = {
#     "type": "line_geo",
#     "lat": lat_long_weight_dict[first_year][0],
#     "lon": lat_long_weight_dict[first_year][1],
#     "mode": "lines",
#     "locationmode": "USA-states",
#     "line": dict(width=1, color="red"),
#     "opacity": lat_long_weight_dict[first_year][2]
# }

# for year in years:
#     frame = {"data": [], "name": str(year)}

#     data_dict = {
#         "type": "line_geo",
#         "lat": lat_long_weight_dict[year][0],
#         "lon": lat_long_weight_dict[year][1],
#         "mode": "lines",
#         "locationmode": "USA-states",
#         "line": dict(width=1, color="red"),
#         "opacity": lat_long_weight_dict[year][2]
#     }

#     frame["data"].append(data_dict)
#     fig_dict["frames"].append(frame)
#     slider_step = {"args": [
#         [year],
#         {"frame": {"duration": 300, "redraw": False},
#          "mode": "immediate",
#          "transition": {"duration": 300}}
#     ],
#         "label": year,
#         "method": "animate"}
    
#     sliders_dict["steps"].append(slider_step)

# fig_dict["layout"]["sliders"] = [sliders_dict]

# fig_test = go.Figure(fig_dict)

# fig_test.update_layout(
#     title_text = 'TESTING',
#     showlegend = False,
#     geo = go.layout.Geo(
#         scope = 'north america',
#         projection_type = 'azimuthal equal area',
#         showland = True,
#         landcolor = 'rgb(243, 243, 243)',
#         countrycolor = 'rgb(204, 204, 204)',
#     ),
# )
# fig_test.show()

In [14]:
# fig_ex = px.line_geo(data, lat="BUYER_LAT", lon="BUYER_LONG",
#                      animation_frame="TRANSACTION_YEAR", hover_name="BUYER_NAME",
#                      projection="azimuthal equal area", scope="usa",
#                      title="Testing...")
# fig_ex.show()

In [15]:
# no_date = data[["REPORTER_NAME", "REPORTER_LAT", "REPORTER_LONG",
#                 "BUYER_NAME", "BUYER_LAT", "BUYER_LONG", "DOSAGE_UNIT"]]

In [16]:
# no_date.head()

In [17]:
# no_date.shape

In [18]:
# data_agg = no_date.groupby(["REPORTER_NAME", "REPORTER_LAT", "REPORTER_LONG",
#                             "BUYER_NAME", "BUYER_LAT", "BUYER_LONG"]).agg({
#                                 "DOSAGE_UNIT": "sum"}).reset_index()

In [19]:
# data_agg.head()

In [20]:
# data_agg["DOSAGE_UNIT"].describe()

In [21]:
# data_agg["DOSAGE_UNIT"].sum()

In [22]:
test_123 = data_agg.head(100)

In [26]:
test_123.head()

,REPORTER_NAME,REPORTER_LAT,REPORTER_LONG,BUYER_NAME,BUYER_LAT,BUYER_LONG,TRANSACTION_YEAR,DOSAGE_UNIT
0,"A F HAUSER, INC",41.5,-87.0,"ADAMS, NATHALIA C (DVM)",33.23,-97.11,2006,5000.0
1,"A F HAUSER, INC",41.5,-87.0,"ALLRED, JOHN A",29.70,-98.10,2006,1900.0
2,"A F HAUSER, INC",41.5,-87.0,"BOBECHKO, WALTER P MD",32.90,-96.79,2006,1500.0
3,"A F HAUSER, INC",41.5,-87.0,"BRANTLEY, GARY C DVM",32.97,-96.74,2009,700.0
4,"A F HAUSER, INC",41.5,-87.0,"BRANTLEY, GARY C DVM",32.97,-96.74,2010,200.0


In [28]:
years

[2006, 2007, 2008, 2009, 2010, 2011, 2012]

In [65]:
# Using the ipywidgets
@interact
# Defining our visualization function
# Here, the init values in the function define the dropdown menus
def track_shipments(year=years):

    # Creating a dataframe based on the year chosen
    df = data_agg.loc[data_agg["TRANSACTION_YEAR"] == year]

    fig = go.Figure()

    fig.add_trace(go.Scattergeo(
        locationmode='USA-states',
        lon=df['BUYER_LONG'],
        lat=df['BUYER_LAT'],
        text=df['BUYER_NAME'],
        hoverinfo='text',
        mode='markers',
        marker=dict(
            size=df["DOSAGE_UNIT"] / 100000,
            color=df["DOSAGE_UNIT"],
            colorscale="Viridis",
            line=dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        )))

    fig.update_layout(
        title_text="Received Opioid Shipments in Texas, " + str(year),
        showlegend=False,
        geo=go.layout.Geo(
            scope='usa',
            projection=go.layout.geo.Projection(
                type='albers usa',
                scale=2.3
            ),
            center={'lat': 31.9686, 'lon': -99.9018},
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',

        )
    )

    fig.show()
    
    offline.plot(fig, filename='opioids_received_TX'+'.html') 

interactive(children=(Dropdown(description='year', options=(2006, 2007, 2008, 2009, 2010, 2011, 2012), value=2…